In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [3]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [28]:
from distutils.dir_util import copy_tree
toDir = "AllTrain"
cats = "/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/train/cat"
copy_tree(cats ,toDir)
dogs = "/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/train/dog"
copy_tree(dogs ,toDir)

['AllTrain/dog.8011.jpg',
 'AllTrain/dog.7322.jpg',
 'AllTrain/dog.1753.jpg',
 'AllTrain/dog.5535.jpg',
 'AllTrain/dog.3144.jpg',
 'AllTrain/dog.775.jpg',
 'AllTrain/dog.11102.jpg',
 'AllTrain/dog.11664.jpg',
 'AllTrain/dog.3622.jpg',
 'AllTrain/dog.5253.jpg',
 'AllTrain/dog.9469.jpg',
 'AllTrain/dog.1035.jpg',
 'AllTrain/dog.6982.jpg',
 'AllTrain/dog.8777.jpg',
 'AllTrain/dog.7444.jpg',
 'AllTrain/dog.1021.jpg',
 'AllTrain/dog.6996.jpg',
 'AllTrain/dog.8763.jpg',
 'AllTrain/dog.7450.jpg',
 'AllTrain/dog.12379.jpg',
 'AllTrain/dog.11670.jpg',
 'AllTrain/dog.4159.jpg',
 'AllTrain/dog.3636.jpg',
 'AllTrain/dog.2528.jpg',
 'AllTrain/dog.5247.jpg',
 'AllTrain/dog.10208.jpg',
 'AllTrain/dog.5521.jpg',
 'AllTrain/dog.761.jpg',
 'AllTrain/dog.3150.jpg',
 'AllTrain/dog.11116.jpg',
 'AllTrain/dog.8005.jpg',
 'AllTrain/dog.7336.jpg',
 'AllTrain/dog.1747.jpg',
 'AllTrain/dog.6028.jpg',
 'AllTrain/dog.991.jpg',
 'AllTrain/dog.5509.jpg',
 'AllTrain/dog.10220.jpg',
 'AllTrain/dog.2266.jpg',
 'AllTra

In [29]:
filenames = []
categories = []
Train_cat = os.listdir("/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/train/cat")
for filename in Train_cat :
        categories.append(1)
filenames.extend(Train_cat )

Train_dog = os.listdir("/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/train/dog")
for filename in Train_dog :
        categories.append(0)
filenames.extend(Train_dog )

train_df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

train_df["category"] = train_df["category"].replace({0: 'dog', 1: 'cat'}) 

In [4]:
# filenames=os.listdir("/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/train")
# categories=[]
# for f_name in filenames:
#     category=f_name.split('.')[0]
#     if category=='dog':
#         categories.append(1)
#     else:
#         categories.append(0)
# df=pd.DataFrame({
#     'filename':filenames,
#     'category':categories
# })

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

2022-12-30 15:00:00.542416: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-30 15:00:00.542751: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

In [7]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [8]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [ ]:
# train_generator = train_datagen.flow_from_dataframe(
#     train_df, "AllTrain/", 
#     x_col='filename',
#     y_col='category',
#     target_size=IMAGE_SIZE,
#     class_mode='categorical',
#     batch_size=batch_size
# )

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(
                                train_df, "AllTrain/", 
                                x_col='filename',
                                y_col='category',
                                target_size=IMAGE_SIZE,
                                class_mode='categorical',
                                batch_size=batch_size
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "AllTrain/", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)
test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)
test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/test1",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode= 'categorical',
                                                 batch_size=batch_size)

In [27]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/train",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/train", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)
test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)
test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/test1",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode= 'categorical',
                                                 batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


/Users/javm/miniforge3/envs/tf-env/lib/python3.8/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 20000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [22]:
# FIX FROM COMMENTS FOR ABOVE CELL
# ----ORIGINAL----
# test_generator = train_datagen.flow_from_dataframe(train_df,
# "./dogs-vs-cats/test/",x_col='filename',y_col='category',
# target_size=Image_Size,
# class_mode='categorical',
# batch_size=batch_size)
# 
# ----FIX----
#  test_generator = train_datagen.flow_from_dataframe(test_df,
# “./dogs-vs-cats/test1/”,x_col=’filename’,y_col=None,
# target_size=Image_Size,
# class_mode=None,
# batch_size=batch_size
# )

In [23]:
epochs=10
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/10


2022-12-30 15:39:38.292876: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x10c259bd0
2022-12-30 15:39:38.292922: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x10c259bd0
2022-12-30 15:39:38.296759: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x10c259bd0
2022-12-30 15:39:38.302492: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x10c259bd0
2022-12-30 15:39:38.309924: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x10c259bd0
2022-12-30 15:39:38.309963: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed a

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_16' defined at (most recent call last):
    File "/Users/javm/miniforge3/envs/tf-env/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/javm/miniforge3/envs/tf-env/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/javm/miniforge3/envs/tf-env/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/javm/miniforge3/envs/tf-env/lib/python3.8/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/Users/javm/miniforge3/envs/tf-env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/javm/miniforge3/envs/tf-env/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_foreve

In [24]:
model.save("model1_catsVSdogs_10epoch.h5")

In [25]:
test_filenames = os.listdir("/Users/javm/Desktop/Cats-Dogs-Classifier/dogs-vs-cats/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [26]:
predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))

ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [22]:
test_df['category'] = np.argmax(predict, axis=-1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

NameError: name 'predict' is not defined

In [23]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./dogs-vs-cats/test1/"+filename, target_size=Image_Size)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

NameError: name 'test_df' is not defined

In [24]:
results={
    0:'cat',
    1:'dog'
}
from PIL import Image
import numpy as np
im=Image.open("__image_path_TO_custom_image")
im=im.resize(Image_Size)
im=np.expand_dims(im,axis=0)
im=np.array(im)
im=im/255
pred=model.predict_classes([im])[0]
print(pred,results[pred])

FileNotFoundError: [Errno 2] No such file or directory: '__image_path_TO_custom_image'